This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [7]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [134]:
# First, import the relevant modules
import requests
import collections


In [26]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-01-02&api_key="
r = requests.get(url+API_KEY)
json_data_day = r.json()
print(json_data_day.items())
print(type(json_data_day))

[(u'dataset_data', {u'column_names': [u'Date', u'Open', u'High', u'Low', u'Close', u'Change', u'Traded Volume', u'Turnover', u'Last Price of the Day', u'Daily Traded Units', u'Daily Turnover'], u'collapse': None, u'end_date': u'2017-01-02', u'transform': None, u'order': None, u'frequency': u'daily', u'limit': None, u'column_index': None, u'data': [[u'2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None]], u'start_date': u'2017-01-01'})]
<type 'dict'>


In [27]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.json()['dataset_data']['column_names'])

[u'Date', u'Open', u'High', u'Low', u'Close', u'Change', u'Traded Volume', u'Turnover', u'Last Price of the Day', u'Daily Traded Units', u'Daily Turnover']


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [30]:
# 1. Collect data from FSE for AFX_X for the whole year 2017
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key="
r = requests.get(url+API_KEY)



In [31]:
# 2. Convert the returned JSON object into a Python dictionary
json_data = r.json()
print(type(json_data))

#this seems to easy... 

<type 'dict'>


In [86]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.

#highest
stock_data = json_data['dataset_data']['data']
type(stock_data)
max_open = None
for i in stock_data:
    if i[1] > max_open: max_open = i[1]

print('The highest opening price was:') 
print(max_open)

#lowest using a list comprehension
opening_prices = [i[1] for i in stock_data if i[1]!= None]
min_open = min(opening_prices)

print('The lowest opening price was:')
print(min_open)

The highest opening price was:
53.11
The lowest opening price was:
34.0


In [ ]:
# 4. What was the largest change in any one day (based on High and Low price)?

In [133]:
#to make finding data more intuitive from here on out, I created nested dictionary  {date:{column_name: data}
column_names = json_data['dataset_data']['column_names']
named_dict = {}
max_change = None
for lists in stock_data:
    named_dict[lists[0]] = dict(zip(column_names, lists))

In [172]:
#used list comprehension to make list of changes
changes = [value['Change'] for key, value in named_dict.items() if value['Change']!= None]
max_change = max(changes, key = abs)

print('The largest change in one day was:')
print(max_change)

The largest change in one day was:
-0.44


In [173]:
#5. What was the largest change between any two days (based on Closing Price)?
#...so does this mean the difference between the closing price one day and the closing price on the next? 

In [210]:
dates_od = collections.OrderedDict(sorted(named_dict.items()))

In [223]:
#create list of closing prices
closing = [value['Close'] for key, value in dates_od.items() if value['Close']!=None]
day_changes = []
for i, value in enumerate(closing):
    if i > 0:
        day_changes.append(closing[i]-closing[i-1])

print('The largest change in closing price over two days was')
print(max(day_changes, key=abs))

The largest change in closing price over two days was
-2.56


In [231]:
# 6.What was the average daily trading volume during this year?
trade_vol = [value['Traded Volume'] for key, value in dates_od.items() if value['Traded Volume']!=None]
avg_vol = sum(trade_vol)/len(trade_vol)
print('The average daily trading volume during 2017 was:')
print(avg_vol)

The average daily trading volume during 2017 was:
89124.3372549


In [237]:
# 7. (Optional) What was the median trading volume during this year. 
#(Note: you may need to implement your own function for calculating the median.)

def median(L):
    L = sorted(L)
    if len(L)%2 != 0:
        mid = (len(L)+1)/2
        M = L[mid]
    else: 
        mid1 = len(L)/2
        mid2 = mid1+1
        M = (L[mid1]+ L[mid2])/2
    return M
print('The median trading volume during 2017 was:')
print(median(trade_vol))

The median trading volume during 2017 was:
76600.0
